**Imports**

In [1]:
import os
from pathlib import Path
import pandas as pd
import yt_dlp

**Download Videos**

In [2]:
def download_video(url, name, output_path, start_time=0, duration=10):
    # Combine output path with filename
    output_path = output_path / f'{name}.mp4'
    
    ydl_opts = {
        'format': 'best',
        'outtmpl': str(output_path),
        'quiet': True,
        'no_warnings': True,
        'download_ranges': lambda info_dict, ydl: [
            {
                'start_time': start_time,
                'end_time': start_time + duration,
            }
        ],
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
            print(f"Downloaded video: {name}")
        return True
    except Exception as e:
        print(f"Error downloading video: {e}")
        return False

**Create Labels**

In [3]:
def add_label(csv_path, url,start_time=0, duration=10, labels=None, division=None, pattern=None):
    # Check if the CSV file exists
    if not os.path.exists(csv_path):
        df = pd.DataFrame(columns=['id', 'url', 'start_time', 'duration', 'labels', 'division', 'pattern'])
        df.to_csv(csv_path, index=False)
    else:
        # Load existing CSV
        df = pd.read_csv(csv_path)

    # Get next ID
    if len(df) == 0:
        next_id = 1
    else:
        next_id = df['id'].max() + 1
    
    # Add new rows
    new_row = {
        'id': next_id,
        'url': url,
        'start_time': start_time,
        'duration': duration,
        'labels': labels,
        'division': division,
        'pattern': pattern,
    }

    # Append new rows
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    # Save updated CSV
    df.to_csv(csv_path, index=False)
    print(f"Added video with ID {next_id} to {csv_path}")

**Batch Download Function**

In [4]:
def download_all_from_csv(csv_path):
    df = pd.read_csv(csv_path)

    for index, row in df.iterrows():
        # Create folder with label name
        os.makedirs(f'../data/raw/videos/{row["labels"]}', exist_ok=True)

        # Download video
        video_name = f"{row['labels']}_{row['id']}.mp4"
        VIDEO_DIR = Path(f'../data/raw/videos/{row["labels"]}')
        
        print(f"Downloading {row['id']}: {row['labels']}...")
        success =download_video(
            row['url'],
            video_name,
            VIDEO_DIR,
            start_time=row['start_time'],
            duration=row['duration'],
        )

In [5]:
os.makedirs('../data', exist_ok=True)
CSV_PATH = '../data/labels.csv'

# Function format: add_label(csv_path, url,start_time=0, duration=10, labels=None, division=None, pattern=None)
add_label(CSV_PATH,'https://www.youtube.com/shorts/dWQqGDk8ygc', 0, 4,'sugar_push','beginner','basic_triple_triple')

# Download all videos
download_all_from_csv(CSV_PATH)

Added video with ID 1 to ../data/labels.csv
Downloaded video: sugar_push_1.mp4                      
